In [1]:
import uuid
import asyncio
import pandas as pd
from tqdm import tqdm
from order_book_simulator.config import OrderRequest, Priority
from order_book_simulator.config import OrderRequest, Side, Priority
from order_book_simulator.core import OrderBook, MatchingEngine
from order_book_simulator.websocket import ConnectionManager
from order_book_simulator.analysis import MetricsCalculator

In [2]:
orders_df = pd.read_csv('../data/orders-1.csv')

In [3]:
# just to adhere to my definition of the OrdrerRequest dataclass
# unix timestamp is more efficient for computation and storage
# trader_uuid is more secure and flexible than group number
# (in real life, traders won't identify themselves with group numbers)
# so we convert them here
orders_df["unix_timestamp"] = pd.to_datetime(orders_df["Timestamp"]).astype(int) // 10**9
group_to_uuid = {group: uuid.uuid4() for group in orders_df["Group Number"].unique()}
orders_df["trader_uuid"] = orders_df["Group Number"].map(group_to_uuid)

In [4]:
order_book = OrderBook()
metrics_calculator = MetricsCalculator(order_book, 0.01)
connection_manager = ConnectionManager()

# websocket related stuff are not required for this assignment but we designed some functions to allow that
matching_engine = MatchingEngine(order_book, 
                                 connection_manager, # not needed for assignment
                                 asyncio.get_event_loop(), # not needed for assignment
                                 metrics_calculator)

In [ ]:
filled_trades = []
total_orders = len(orders_df)
checkpoints = {4, 19, total_orders - 1} 

for index, row in tqdm(orders_df.iterrows(), total=total_orders, desc="Processing orders"):
    order_request = OrderRequest(
        trader_id = row["trader_uuid"],
        side = row["Buy Sell Direction"],
        priority=Priority.HIGH,
        timestamp=row["unix_timestamp"],
        price=row["Order Price"],
        quantity=row["Quantity"]
    )
    trade = matching_engine.process_order(order_request)
    if trade:
        filled_trades.extend(trade)
    
    if index in checkpoints:
        label = "FINAL" if index == total_orders - 1 else f"#{index + 1}"
        print(f"\n=== Order Book after {label} orders ===")
        print(order_book)

# Print all executed trades
print("\n=== ALL EXECUTED TRADES ===")
print(f"Total number of trades: {len(filled_trades)}\n")
for i, trade in enumerate(filled_trades, 1):
    print(f"Trade {i}: Quantity: {trade.quantity} @ Price: {trade.price} | "
          f"Maker: {trade.maker_order_id} | Taker: {trade.taker_order_id} | "
          f"Side: {trade.taker_side.value}")

Processing orders:   0%|          | 0/248 [00:00<?, ?it/s]


=== Order Book after #5 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           20 │     12.00 │           30 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after #20 orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     11.00 │           50 │     12.00 │           55 │
│     10.00 │           75 │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘



=== Order Book after FINAL orders ===


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Bid Price ┃ Bid Quantity ┃ Ask Price ┃ Ask Quantity ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│     10.00 │          190 │     11.00 │           81 │
│      9.00 │          228 │     12.00 │          510 │
│      8.00 │           25 │     13.00 │          155 │
│      7.00 │          130 │     15.00 │           50 │
│           │              │     50.00 │           11 │
└───────────┴──────────────┴───────────┴──────────────┘

Processing orders: 100%|██████████| 248/248 [00:00<00:00, 11817.49it/s]



=== ALL EXECUTED TRADES ===
Total number of trades: 204

Trade 1: 30@11.0 | Maker: 33905946-52f6-4e78-9f0b-a96e5d9b6a4a | Taker: 20fb9367-ee44-43fd-b30d-4ea5f5deb558 | Side: sell
Trade 2: 30@11.0 | Maker: 33905946-52f6-4e78-9f0b-a96e5d9b6a4a | Taker: afaee8d7-d388-40cf-b0f8-38f39713209a | Side: sell
Trade 3: 10@11.0 | Maker: 33905946-52f6-4e78-9f0b-a96e5d9b6a4a | Taker: 6cf75ba1-3006-4014-b4f5-336d1381dfdc | Side: sell
Trade 4: 10@11.0 | Maker: 33905946-52f6-4e78-9f0b-a96e5d9b6a4a | Taker: 6dd5718a-cc8b-4bf7-ad7e-cf4d1564894c | Side: sell
Trade 5: 50@10.0 | Maker: 3dcedd43-2c53-40a0-b58a-ccbf69a5288b | Taker: 9e7d7111-9d94-48da-814b-c3d54637f98c | Side: sell
Trade 6: 30@10.0 | Maker: 3dcedd43-2c53-40a0-b58a-ccbf69a5288b | Taker: 1447c37f-6691-4948-bd84-06e437cb9858 | Side: sell
Trade 7: 20@11.0 | Maker: 874c7d0d-08c4-4050-b796-169098a4c10d | Taker: 1f0ae803-62c4-4a46-ac88-a47a8a066b09 | Side: buy
Trade 8: 5@12.0 | Maker: 788b6cda-cbd6-48a5-8af1-5c905354cd36 | Taker: 1f0ae803-62c4-4a4

In [6]:
len(metrics_calculator.spread_bps)
print(f"\nNumber of trades executed: {len(filled_trades)}")
print(f"Number of spread snapshots taken: {len(metrics_calculator.spread_bps)}")


Number of trades executed: 204
Number of spread snapshots taken: 247


In [7]:
len(metrics_calculator.spread_ticks)
print(f"Number of spread snapshots taken: {len(metrics_calculator.spread_ticks)}")
spread_ticks_list = metrics_calculator.spread_ticks

Number of spread snapshots taken: 247


In [8]:
final_metrics = metrics_calculator.calculate_averages(orders_df["unix_timestamp"].max())

--- Final Metrics ---

Simple Average (Arithmetic Mean):

Simple Avg Bid-Ask Spread (in Ticks): 106.4777

Simple Avg Bid-Ask Spread (in Basis Points): 1055.2239

Simple Avg Best-Level Market Depth (in Dollars): $2,635.57

...

Time-Weighted Average:

Time-Weighted Avg Bid-Ask Spread (in Ticks): 93.1985

Time-Weighted Avg Bid-Ask Spread (in Basis Points): 897.4538

Time-Weighted Avg Best-Level Market Depth (in Dollars): $2,421.82

Time-Weighted Avg Order Imbalance: +0.3439